In [4]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import torch
from model import FloodDataPipeline, QuantumFloodCircuit, QGraphPredictor  # Replace with your module import

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
# Load real flood data
pipeline = FloodDataPipeline()
sample_graphs = pipeline.load_flood_data("FloodGNN-GRU/train.npz")
sample = sample_graphs[0]

In [ ]:
# Convert to NetworkX graph
G = pipeline.to_networkx_graph(sample)
pos = nx.spring_layout(G, seed=42)

In [ ]:
# Visualize water depth at selected time points
time_points = [0, 5, 10]
water_depth = sample.dynamic[:, :, 0].numpy()

fig, axes = plt.subplots(1, len(time_points), figsize=(18, 6))
for idx, t in enumerate(time_points):
    ax = axes[idx]
    wd_t = water_depth[:, t]
    nx.draw(
        G, pos, node_color=wd_t, node_size=50, edge_color='gray',
        cmap=plt.cm.Blues, ax=ax, with_labels=False
    )
    ax.set_title(f"Water Depth at t={t}")
plt.tight_layout()
plt.show()

In [ ]:
# Initialize quantum model and transparency engine
num_features = sample.x.shape[1] + sample.dynamic.shape[2]
quantum_circuit = QuantumFloodCircuit(num_features, sample.num_edges, active_edges=[])
predictor = QGraphPredictor(quantum_circuit)
qvism = QVISMTransparencyEngine(predictor)

In [ ]:
# Generate transparency report
time_step = 5
report = qvism.generate_transparency_report(sample, time_step)
print("Transparency Report Generated:")
print(report)

In [ ]:
# Plot Hilbert space overlap
hilbert_overlap = report["hilbert_space_overlap"]
plt.figure(figsize=(6, 4))
plt.bar(["Hilbert Overlap"], [hilbert_overlap], color='purple')
plt.title("Hilbert Space Overlap at t=5")
plt.ylabel("Overlap")
plt.ylim(0, 1)
plt.show()

In [ ]:
# Plot edge influence
edge_influences = report["edge_influences"]
plt.figure(figsize=(8, 4))
plt.bar(edge_influences.keys(), edge_influences.values(), color='teal')
plt.title("Edge Influence at t=5")
plt.xlabel("Edge Index")
plt.ylabel("Influence")
plt.show()

In [ ]:

# Plot node sensitivity
node_sensitivities = report["node_sensitivities"]
plt.figure(figsize=(8, 4))
plt.bar(node_sensitivities.keys(), node_sensitivities.values(), color='orange')
plt.title("Node Sensitivity at t=5")
plt.xlabel("Node Index")
plt.ylabel("Sensitivity")
plt.show()

In [ ]:
# Print summary
print("Transparency Report Summary:")
print(f"Time Step: {report['timestamp']}")
print(f"Average Flood Risk: {report['average_flood_risk']:.4f}")
print(f"Risk Level: {report['risk_level']}")
print(f"Active Edges: {report['active_edges']}")
print(f"Hilbert Space Overlap: {report['hilbert_space_overlap']:.4f}")